In [1]:

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import emoji
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack, csr_matrix, vstack
import pickle
analyzer_emoji = SentimentIntensityAnalyzer()

In [ ]:
data = pd.read_csv('sentimentdata.csv')
data.drop('Unnamed: 0',axis=1,inplace=True)
data = data.drop(["tweet_id", "user_id", "user_screen_name","user_name","created_at","static_link","retweets","favorites","Tag"], axis=1)
data.sentiment = data.sentiment.map({'positive':0,'negative':1})
data.head()

/home/busra/PycharmProjects/Piton/myenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Using TensorFlow backend.


2.3.1


In [2]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [55]:
dataset = pd.read_csv('data.csv')
dataset.head()

,Yorum,Duygu
0,Çok iyi. Tesekkurler..,1
1,çok iyi ve hızlı bir ürün,1
2,teşekkürler...,1
3,Gerçekten kullanmasını bilene çok iyi orta seg...,1
4,Kargo yavaş gelmedi şansıma 3 gün sonra yola ç...,1


In [56]:
# get all labels and reviews as a list
target = dataset['Duygu'].values.tolist()
data = dataset['Yorum'].values.tolist()

In [57]:
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [58]:

dataset.shape

(976, 2)

In [59]:
# We will get 10000 most frequently used words in our dataset
num_words = 10000

# Define tokenizer with Keras...
# If we don't define num_words, then we use all words in our dataset.
tokenizer = Tokenizer(num_words=num_words)

In [60]:
# Now tokenize the data
tokenizer.fit_on_texts(data)

In [61]:
# saving tokenizer
import pickle

with open('turkish_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [62]:
# load tokenizer
with open('turkish_tokenizer.pickle', 'rb') as handle:
    turkish_tokenizer = pickle.load(handle)

In [63]:
"""
Tokenize our training data. This will only works for the words in our 10000 tokenizer. 
If a word is not in 10000 tokenized words, it will be ignored.
"""
x_train_tokens = turkish_tokenizer.texts_to_sequences(x_train)

In [64]:
x_train[100]

'ceptelefonuonline satıcısından aldım gayet hızlı ve doğru geldi kontrol ettim kayıtlı teşekkürler '

In [65]:

x_train_tokens[100]

[2917, 979, 20, 11, 31, 3, 412, 42, 2918, 343, 2919, 32]

In [66]:

x_test_tokens = turkish_tokenizer.texts_to_sequences(x_test)
print(x_test_tokens)

[[30, 238, 2234, 2235, 296, 2236, 2237, 2238, 2239, 1135, 2240, 1361, 2241, 75, 805, 449, 2242, 398, 2243, 2244, 4, 2245, 2246, 2247, 14, 228, 2248, 2249, 2250, 799], [663, 136, 4, 1428, 128, 22, 63, 1, 29, 78, 586, 4, 423, 2251, 69, 2252], [58, 160, 523], [4, 148, 24, 97, 1488, 155, 924, 5, 2253, 214, 41, 90, 27, 25, 288, 54, 440, 3, 145, 77, 86, 503, 179, 418, 333, 2254, 3, 183, 418, 1136, 745, 1, 4, 2255, 49, 2256, 92, 2257, 74, 257, 2258, 108, 2259, 1463, 2260, 134, 1, 261, 574, 14, 341, 925, 274, 1, 2261, 14, 341, 108, 925, 1540, 274, 1, 1115, 2262, 1336, 3, 1137, 533, 757], [118, 40, 27, 641, 322, 1541, 20, 13, 154, 241, 146, 183, 418, 39, 2263, 724, 524, 2264, 2265, 939, 2, 423, 421, 267, 34, 109, 267, 3, 138, 30, 314, 756, 349, 70, 1508, 498, 94, 1524, 569, 18, 183, 232, 2266, 14, 1464, 478, 1542, 2267, 3, 515, 191, 5, 2268, 417, 2269, 2270, 2271, 327, 1020, 94, 2272, 2273, 1327, 2274, 926, 1138, 359, 763, 2275, 1011, 1127, 65, 56, 66, 548, 1537, 45, 410, 185, 33, 235, 1, 175, 

In [67]:
"""
We will add Padding for our comments.
In RNN, we give predefined-sized inputs. But our comments consist of different sized inputs, so we need to define
a input size for comments. If size > comment, then add 0s for the gap, otherwise trim the comment.
"""

# How many tokens in each comment?
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]

# Convert list to numpy array
num_tokens = np.array(num_tokens)
num_tokens.shape

(976,)

In [18]:
# In average, how many tokens in one comment?
np.mean(num_tokens)

30.940573770491802

In [19]:
# Max token amount?
np.max(num_tokens)

307

In [20]:
# Index of max token
np.argmax(num_tokens)

457

In [68]:
# Define max tokens for all comments
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) #returns float
max_tokens = int(max_tokens)
max_tokens

94

In [69]:
# How many tokens are smaller than max_tokens?
np.sum(num_tokens < max_tokens) / len(num_tokens) * 100  # output: 96%. which means we will only lose info in 4%.

95.1844262295082

In [70]:
# Let's add paddings... So, all datas will be in the same size.
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [42]:
# Check the sizes
print(x_train_pad.shape)
print(x_test_pad.shape)

(780, 94)
(196, 94)


In [71]:
# In Keras, we can get tokens from strings but not vice versa.
# So we will write a function to get strings from tokens
idx = turkish_tokenizer.word_index
#print(idx)
# in idx, key value pair like 'çok': 1. But we want it reverse.
inverse_map = dict(zip(idx.values(), idx.keys()))
first_five = {k: inverse_map[k] for k in sorted(inverse_map.keys())[:5]}
first_five

{1: 'bir', 2: 'çok', 3: 've', 4: 'telefon', 5: 'bu'}

In [72]:
def convert_tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words)
    return text

In [73]:
model = Sequential()


In [74]:
embedding_size = 50 # vector with 50 size for every word

In [76]:
"""
Now we will create an embedding layer in Keras.
We won't use word2vec or glove, instead we create word vectors randomly.
"""

# Add embedding layer to our model.
# embedding matris size = num_words * embedding_size -> 10.000 * 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [77]:
# 3-layered LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
model.add(Dense(1, activation='sigmoid'))

In [78]:

from tensorflow.python.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(lr=1e-3)

In [79]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])

In [80]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 94, 50)            500000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 94, 16)            4288      
_________________________________________________________________
dropout_4 (Dropout)          (None, 94, 16)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 94, 8)             800       
_________________________________________________________________
dropout_5 (Dropout)          (None, 94, 8)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dropout_6 (Dropout)          (None, 4)                

In [85]:
# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train_pad, y_train, epochs=10, batch_size=100)

Epoch 1/10
780/780 [==============================] - 1s 1ms/step - loss: 0.1205 - accuracy: 0.9808
Epoch 2/10
780/780 [==============================] - 1s 1ms/step - loss: 0.1148 - accuracy: 0.9872
Epoch 3/10
780/780 [==============================] - 1s 1ms/step - loss: 0.1029 - accuracy: 0.9910
Epoch 4/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0984 - accuracy: 0.9897
Epoch 5/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0973 - accuracy: 0.9897
Epoch 6/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0958 - accuracy: 0.9910
Epoch 7/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0854 - accuracy: 0.9910
Epoch 8/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0836 - accuracy: 0.9910
Epoch 9/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0789 - accuracy: 0.9910
Epoch 10/10
780/780 [==============================] - 1s 1ms/step - loss: 0.0753 - accuracy: 0.9936

In [86]:

result = model.evaluate(x_test_pad, y_test)
result

196/196 [==============================] - 0s 947us/step


[2.410176542340493, 0.2091836780309677]

In [87]:
accuracy = (result[1]) * 100
accuracy

20.91836780309677

In [88]:
text1 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text2 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem :("
text3 = "hiç resimde gösterildiği gibi değil..."
text4 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text5 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim. ürünü iade ediyorum"
text6 = "tam bir fiyat performans ürünü"
text7 = "güzel bir ürün değil"
texts = [text1, text2,text3,text4,text5,text6,text7]

In [89]:
tokens = turkish_tokenizer.texts_to_sequences(texts)

In [90]:
tokens = turkish_tokenizer.texts_to_sequences(texts)
tokens

[[97, 1, 5384, 1858, 5, 21, 5, 4519],
 [229, 62, 259, 64, 2, 286, 42, 3, 21, 16, 764],
 [69, 44, 45],
 [86, 1113, 3559, 259, 410, 104, 740, 32],
 [69, 5, 46, 86, 1, 781, 888, 38, 505, 96],
 [53, 1, 26, 36, 38],
 [7, 1, 21, 45]]

In [91]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)

In [93]:
model.predict(tokens_pad)

array([[0.9695683 ],
       [0.8484215 ],
       [0.9224933 ],
       [0.6162714 ],
       [0.68721837],
       [0.9622597 ],
       [0.96230775]], dtype=float32)

In [ ]:
def convert(texts):
    tokens = turkish_tokenizer.texts_to_sequences(texts)
    tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
    return model.predict(tokens_pad)[0][0]